In [1]:
import numpy as np
import pandas as pd
import sys, os, json, csv

https://covid.saude.gov.br/

https://github.com/leogermani/estados-e-municipios-ibge/blob/master/estados.csv



In [17]:
data = '../data'
df = pd.read_csv(f'{data}/brasil.csv', sep=';', encoding='utf-8')
df.columns = ['região'] + df.columns.tolist()[1:]
df.
df.head()

,região,sigla,data,casosNovos,casosAcumulados,obitosNovos,obitosAcumulados
0,Norte,RO,30/01/2020,0,0,0,0
1,Norte,RO,31/01/2020,0,0,0,0
2,Norte,RO,01/02/2020,0,0,0,0
3,Norte,RO,02/02/2020,0,0,0,0
4,Norte,RO,03/02/2020,0,0,0,0


In [33]:
estados = pd.read_csv(f'{data}/estados.csv')
estados.loc[0]['COD'], estados.loc[0]['NOME'], estados.loc[0]['SIGLA']

(35, 'São Paulo', ' SP')

padding

In [35]:
def remove_padding(row):
    row['SIGLA'] = row['SIGLA'][1:]
    return row

estados = estados.apply(remove_padding, axis=1)
estados = estados.set_index('SIGLA')
estados.head()

,COD,NOME
SIGLA,,
SP,35,São Paulo
PR,41,Paraná
SC,42,Santa Catarina
RS,43,Rio Grande do Sul
MS,50,Mato Grosso do Sul


In [38]:
estados.loc['RO']

COD           11
NOME    Rondônia
Name: RO, dtype: object

In [46]:
def completa_info(row, estados_df):
    estado = estados_df.loc[row['sigla']]
    row['estado'] = estado['NOME']
    row['cod_estado'] = estado['COD']
    row['data'] = '-'.join(row['data'].split('/')[::-1])
    return row

df = df.apply(completa_info, estados_df=estados, axis=1)
df.head()

,região,sigla,data,casosNovos,casosAcumulados,obitosNovos,obitosAcumulados,estado,cod_estado
0,Norte,RO,2020-01-30,0,0,0,0,Rondônia,11
1,Norte,RO,2020-01-31,0,0,0,0,Rondônia,11
2,Norte,RO,2020-02-01,0,0,0,0,Rondônia,11
3,Norte,RO,2020-02-02,0,0,0,0,Rondônia,11
4,Norte,RO,2020-02-03,0,0,0,0,Rondônia,11


Casos de recuperação

In [7]:
df.sort_values(['data'], inplace=True)

falar do range de datas

In [9]:
df.loc[0]['data'], df.loc[len(df) - 1]['data']

('30/01/2020', '09/04/2020')

gerar json

In [50]:
states = df['estado'].unique()
days_before = [0] * 8
dates_before = ['2020-01-{}'.format(n) for n in range(22, 30)]
print(dates_before)

['2020-01-22', '2020-01-23', '2020-01-24', '2020-01-25', '2020-01-26', '2020-01-27', '2020-01-28', '2020-01-29']


In [62]:
covid_jsons = []
for state in states:
    ts_segment = df[df['estado'] == state].copy().reset_index()
    
    covid_json = {
        'estado': state,
        'cod_estado': int(ts_segment.loc[0]['cod_estado']),
        'dates': dates_before + ts_segment['data'].tolist(),
        'confirmed': days_before + ts_segment['casosAcumulados'].tolist(),
        'deaths': days_before + ts_segment['obitosAcumulados'].tolist(),
        'recovered': [0] * (8 + len(ts_segment))
    }
    covid_jsons.append(covid_json)

In [64]:
with open(f'{data}/brasil_covid.json', 'w', encoding='utf-8') as f:
    json.dump(covid_jsons, f, ensure_ascii=False)